In [20]:
from ultralytics import YOLO
import pandas as pd
import numpy as np
from pathlib import Path
import cv2, os

# --------------------
# Define the model
# --------------------
model = YOLO("/home/debasish/Documents/YOLOv8/weights/pose/yolov8x-pose-832.pt")

# Input file or directory
source = Path("/home/debasish/Documents/YOLOv8/videos/image053_10fps.mp4")

# --------------------
# Setup output directories
# --------------------
base_out_dir = Path("outputs/pose")
base_out_dir.mkdir(parents=True, exist_ok=True)

# Create a new run folder (run1, run2, ...)
existing_runs = [d for d in base_out_dir.iterdir() if d.is_dir() and d.name.startswith("run")]
run_num = len(existing_runs) + 1
run_dir = base_out_dir / f"run{run_num}"
run_dir.mkdir(parents=True, exist_ok=True)

print(f"Outputs will be saved in: {run_dir}")

# --------------------
# Collect input files
# --------------------
if source.is_dir():
    files = list(source.glob("*.*"))  # process all files in folder
else:
    files = [source]

# video_exts = [".mp4", ".avi", ".mov", ".mkv"]
video_exts = [".mp4"]
image_exts = [".jpg", ".jpeg", ".png", ".bmp"]

POSE_CONF_THRESHOLD = 0.8  # reject detections below this avg confidence

# --------------------
# Process each file
# --------------------
for file_path in files:
    ext = file_path.suffix.lower()
    if ext not in video_exts + image_exts:
        continue  # skip non-media files

    filename = file_path.stem

    out_video_path = run_dir / f"{filename}.mp4"
    out_img_path = run_dir / f"{filename}.jpg"
    out_csv_path = run_dir / f"{filename}.csv"

    print(f"\nProcessing: {file_path.name}")

    # --------------------
    # Run YOLO
    # --------------------
    if ext in video_exts:
        # Use tracking for videos
        results = model.track(str(file_path),persist=True,show=False,show_labels=False,
                                                show_conf=False,tracker="botsort.yaml",conf=0.5)
    else:
        # Use prediction for images
        results = model.predict(str(file_path),show=False,save=False,conf=0.5)

    # Initialize tracking variables
    frame = 0
    frames = []
    obj_ids = []
    obj_clss = []
    x = []
    y = []
    w = []
    h = []
    key_pts = []

    # --------------------
    # Video setup
    # --------------------
    is_video = ext in video_exts
    if is_video:
        cap = cv2.VideoCapture(str(file_path))
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Fix codec issue: use MP4 container with H.264-compatible codec
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = round(fps) if fps > 0 else 30
        writer = cv2.VideoWriter(str(out_video_path), fourcc, fps, (width, height))

        if not writer.isOpened():
            raise RuntimeError(f"Failed to open VideoWriter: {out_video_path}")
        print(f"VideoWriter opened: {out_video_path}")
    else:
        writer = None

    # --------------------
    # Frame loop
    # --------------------
    for result in results:
        if is_video:
            ret, frame_img = cap.read()
            if not ret:
                break
        else:
            frame_img = cv2.imread(str(file_path))
            if frame_img is None:
                raise RuntimeError(f"Failed to read image: {file_path}")

        if result.boxes is None or result.keypoints is None:
            if writer:
                writer.write(frame_img)
            continue

        boxes = result.boxes.cpu().numpy()
        keypoints = result.keypoints.cpu().numpy()

        if boxes is None or len(boxes) == 0 or keypoints is None or keypoints.xy is None or len(keypoints.xy) == 0:
            if writer:
                writer.write(frame_img)
            continue

        if boxes.cls is None or boxes.xywh is None:
            if writer:
                writer.write(frame_img)
            continue

        # When tracking (videos), ID exists; when predicting (images), assign 0
        obj_cls = boxes.cls
        obj_id = getattr(boxes, "id", None)
        if obj_id is None:
            obj_id = np.zeros_like(obj_cls)
        xywh = boxes.xywh
        kpts_xy = keypoints.xy
        kpts_conf = keypoints.conf

        frame += 1
        for i, j, k, l, c in zip(obj_cls, obj_id, xywh, kpts_xy, kpts_conf):
            avg_conf = np.mean(c)
            if avg_conf < POSE_CONF_THRESHOLD:
                continue  # skip low-confidence detections

            obj_id_int = int(j)
            frames.append(frame)
            obj_clss.append(int(i))
            obj_ids.append(obj_id_int)
            x.append(k[0])
            y.append(k[1])
            w.append(k[2])
            h.append(k[3])
            key_pts.append(l.reshape(-1))
            # --- Draw keypoints ---
            num_kpts = l.shape[0]
            for idx, (px, py) in enumerate(l.astype(int)):
                r = 0
                g = int(255 * (idx / (num_kpts - 1))) if num_kpts > 1 else 255
                b = int(255 * (1 - idx / (num_kpts - 1))) if num_kpts > 1 else 0
                color = (b, g, r)
                cv2.circle(frame_img, (px, py), 3, color, -1)

            # px, py = int(l[0][0]) + 10, int(l[0][1]) + 10
            px, py = int(k[0])+10, int(k[1])+10
            cv2.putText(frame_img, f"ID:{obj_id_int}", (px, py), cv2.FONT_HERSHEY_SIMPLEX, 
                                            0.6, (0,0,0), 2, cv2.LINE_AA)
            

        if writer:
            writer.write(frame_img)
        else:
            cv2.imwrite(str(out_img_path), frame_img)

    # --------------------
    # Cleanup
    # --------------------
    if is_video:
        cap.release()
        writer.release()
        print(f"Video saved at: {out_video_path}")
    else:
        print(f"Image saved at: {out_img_path}")

    # --------------------
    # Save CSV
    # --------------------
    if len(frames) > 0:
        data = {"frame": frames, "obj_cls": obj_clss, "obj_id": obj_ids}
        bbox_out = {"x": x, "y": y, "w": w, "h": h}
        kpts_out1 = pd.DataFrame(data)
        kpts_out2 = pd.DataFrame(bbox_out)
        kpts_out3 = pd.DataFrame(key_pts)
        out = pd.concat([kpts_out1, kpts_out2, kpts_out3], axis=1)
        out.to_csv(out_csv_path, index=False)
        print(f"CSV saved at: {out_csv_path}")
    else:
        print("No valid detections found; skipping CSV save.")

Outputs will be saved in: outputs/pose/run21

Processing: image053_10fps.mp4

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/305) /home/debasish/Documents/YOLOv8/videos/image053_10fps.mp4: 480x832 9 worms, 19.8ms
video 1/1 (frame 2/305) /home/debasish/Documents/YOLOv8/videos/image053_10fps.mp4: 480x832 9 worms, 21.1ms
video 1/1 (frame 3/305) /home/debasish/Documents/YOLOv8/videos/image053_10fps.mp4: 480x832 9 worms, 20.5ms
video 1/1 (frame 4/305) /home/debasish/Documents/YOLOv8/vi